In [1]:
import numpy as np
import random
twopi = 2.*np.pi
oneOver2Pi = 1./twopi

In [2]:
import time

def time_usage(func):
    def wrapper(*args, **kwargs):
        beg_ts = time.time()
        retval = func(*args, **kwargs)
        end_ts = time.time() 
        print("elapsed time: %f" % (end_ts - beg_ts))
        return retval
    return wrapper

In [43]:
#
# For the jam multiruns
# [iso, D, T, X, U, L]

mode = "edge_3"
runs = {1:"edge_3_7.00", 0:"edge_3_14.00"}
in_dir = "/home/walterms/project/walterms/mcmd/output/scratch/"+mode+"/"
trn_dir = "/home/walterms/project/walterms/mcmd/nn/data/train/"
test_dir = "/home/walterms/project/walterms/mcmd/nn/data/test/"
unlabeled_dir = "/home/walterms/project/walterms/mcmd/nn/data/unlbl/"

jidx = np.arange(2,18)
testidxs = np.arange(0,2) # want 400 ea
nblSkip = 1 # Skip first image

# noiseLvl: sigma of Gaussian in units of rod length
rodlen = 1.0
noiseLvl = 0.00*rodlen
thnoise = 0.00
noiseappend = ""
if noiseLvl > 0.0:
    noiseappend = "_"+str(noiseLvl)

In [15]:
processTrain(noise=noiseLvl)

processing edge_3_14.00_2 for training data
processing edge_3_14.00_3 for training data
processing edge_3_14.00_4 for training data
processing edge_3_14.00_5 for training data
processing edge_3_14.00_6 for training data
processing edge_3_14.00_7 for training data
processing edge_3_14.00_8 for training data
processing edge_3_14.00_9 for training data
processing edge_3_14.00_10 for training data
processing edge_3_14.00_11 for training data
processing edge_3_14.00_12 for training data
processing edge_3_14.00_13 for training data
processing edge_3_14.00_14 for training data
processing edge_3_14.00_15 for training data
processing edge_3_14.00_16 for training data
processing edge_3_14.00_17 for training data
Done processing training files
processing edge_3_7.00_2 for training data
processing edge_3_7.00_3 for training data
processing edge_3_7.00_4 for training data
processing edge_3_7.00_5 for training data
processing edge_3_7.00_6 for training data
processing edge_3_7.00_7 for training data

In [8]:
@time_usage
def processTrain(noise=0.):
    for lbl in runs:
        name = runs[lbl] 
        trnlim = -1
        trnfnames = [name+"_"+str(i) for i in jidx]
        fout = open(trn_dir+name+noiseappend,'w') #erases file
        fout.close()
        for f in trnfnames:
            fin = open(in_dir+f,'r')
            print "processing " + f + noiseappend + " for training data"
            fout = open(trn_dir+name+noiseappend,'a')

            # find width from file header
            width, height = 0., 0.
            l = fin.readline().split("|")
            for ll in l:
                if "boxEdge" in ll:
                    width = float(ll.split()[1])
            height = width
            fin.seek(0)

            if width == 0.:
                # calculate edge length based on vertices of first block
                block = []
                for line in fin.readlines():
                    if line == "\n": break
                    if line[0].isalpha(): continue
                    block.append(line)
                fin.seek(0)
                width, height = edgeLenCalc(block)

            if not (fin.readline()[0].isalpha()): fin.seek(0)

            thNorm = oneOver2Pi
            normX, normY = 1./width, 1./height # normalize x and y

            nbl = 0
            fRot = 0. # rotation factor: 0,1,2,3. Multiplied by pi/2
            block = []
            for line in fin.readlines():
                if line == "\n":
                    if nbl < nblSkip:
                        nbl+=1
                        block = []
                        continue
                    fRot = random.randint(0,3)
                    for l in block:
                        fout.write('%f %f %f\n' % (l[0], l[1], l[2]))
                    fout.write('label %f\n\n' % (lbl))
                    block = []
                    nbl+=1
                    continue

                rndxy = [0.,0.]
                rndth = 0.
                if noise > 0.:
                    # Gen three random numbers
                    rndxy = np.random.normal(0,noise,2)
                    rndth = np.random.normal(0,twopi*thnoise,1)
    #                 rndxy = [0.,0.]
    #                 rndth = 0.

                spt = [float(x) for x in line.split()]
                x,y,th = spt[2],spt[3],spt[4]
                # Rotate block
                # note thetas should be [0,2pi] initially
                th_ = fRot*twopi*0.25
                th += th_ + rndth
                if th > twopi: th-=twopi
                th *= thNorm

                x = np.cos(th_)*spt[2] - np.sin(th_)*spt[3] + rndxy[0]
                y = np.sin(th_)*spt[2] + np.cos(th_)*spt[3] + rndxy[1]
                # shift and normalize
                x *= normX
                y *= normY

                block.append([x,y,th])

            fout.close()
            fin.close()
    print "Done processing training files" 

In [21]:
r = np.random.normal(0,noiseLvl,2)
r[0]

0.00013795753618450133

In [16]:
processTest()

processing edge_3_14.00_0 for testing data
processing edge_3_14.00_1 for testing data
Done processing testing files
processing edge_3_7.00_0 for testing data
processing edge_3_7.00_1 for testing data
Done processing testing files
elapsed time: 11.643448


In [10]:
@time_usage
def processTest():
    for lbl in runs:
        name = runs[lbl]
        testfnames = [name+"_"+str(i) for i in testidxs]
        fout = open(test_dir+name,'w') #erases file
        fout.close()
        for f in testfnames:
            fin = open(in_dir+f,'r')
            print "processing " + f + " for testing data"
            fout = open(test_dir+name,'a')

            # find width from file header
            width, height = 0., 0.
            l = fin.readline().split("|")
            for ll in l:
                if "boxEdge" in ll:
                    width = float(ll.split()[1])
            height = width
            fin.seek(0)

            if width == 0.:
                # calculate edge length based on vertices of first block
                block = []
                for line in fin.readlines():
                    if line == "\n": break
                    if line[0].isalpha(): continue
                    block.append(line)
                fin.seek(0)
                width, height = edgeLenCalc(block)

            if not (fin.readline()[0].isalpha()): fin.seek(0)

            thNorm = oneOver2Pi
            normX, normY = 1./width, 1./height # normalize x and y

            nbl = 0
            fRot = 0. # rotation factor: 0,1,2,3. Multiplied by pi/2
            block = []
            for line in fin.readlines():
                if line == "\n":
                    if nbl < 1:
                        nbl+=1
                        block = []
                        continue
                    fRot = random.randint(0,3)
                    for l in block:
                        fout.write('%f %f %f\n' % (l[0], l[1], l[2]))
                    fout.write('label %f\n\n' % (lbl))
                    block = []
                    nbl+=1
                    continue

                spt = [float(x) for x in line.split()]
                x,y,th = spt[2],spt[3],spt[4]
                # Rotate block
                # note thetas should be [0,2pi] initially
                th_ = fRot*twopi*0.25
                th += th_
                if th > twopi: th-=twopi
                th *= thNorm

                x = np.cos(th_)*spt[2] - np.sin(th_)*spt[3]
                y = np.sin(th_)*spt[2] + np.cos(th_)*spt[3]
                # shift and normalize
                x *= normX
                y *= normY

                block.append([x,y,th])

            fout.close()
            fin.close()
    print "Done processing testing files" 

In [44]:
edges = []
ein = open("/home/walterms/mcmd/edge_3",'r')
for line in ein.readlines():
    edges.append(float(line))
unlblnames = [mode+"_"+"%.2f"%(e) for e in edges]
uidx = np.arange(0,18)

In [52]:
processUnlbl()

processing edge_3_14.00_0 for training data
processing edge_3_14.00_1 for training data
processing edge_3_14.00_2 for training data
processing edge_3_14.00_3 for training data
processing edge_3_14.00_4 for training data
processing edge_3_14.00_5 for training data
processing edge_3_14.00_6 for training data
processing edge_3_14.00_7 for training data
processing edge_3_14.00_8 for training data
processing edge_3_14.00_9 for training data
processing edge_3_14.00_10 for training data
processing edge_3_14.00_11 for training data
processing edge_3_14.00_12 for training data
processing edge_3_14.00_13 for training data
processing edge_3_14.00_14 for training data
processing edge_3_14.00_15 for training data
processing edge_3_14.00_16 for training data
processing edge_3_14.00_17 for training data
processing edge_3_13.59_0 for training data
processing edge_3_13.59_1 for training data
processing edge_3_13.59_2 for training data
processing edge_3_13.59_3 for training data
processing edge_3_13.59_

processing edge_3_11.03_5 for training data
processing edge_3_11.03_6 for training data
processing edge_3_11.03_7 for training data
processing edge_3_11.03_8 for training data
processing edge_3_11.03_9 for training data
processing edge_3_11.03_10 for training data
processing edge_3_11.03_11 for training data
processing edge_3_11.03_12 for training data
processing edge_3_11.03_13 for training data
processing edge_3_11.03_14 for training data
processing edge_3_11.03_15 for training data
processing edge_3_11.03_16 for training data
processing edge_3_11.03_17 for training data
processing edge_3_10.82_0 for training data
processing edge_3_10.82_1 for training data
processing edge_3_10.82_2 for training data
processing edge_3_10.82_3 for training data
processing edge_3_10.82_4 for training data
processing edge_3_10.82_5 for training data
processing edge_3_10.82_6 for training data
processing edge_3_10.82_7 for training data
processing edge_3_10.82_8 for training data
processing edge_3_10.82_

processing edge_3_9.38_12 for training data
processing edge_3_9.38_13 for training data
processing edge_3_9.38_14 for training data
processing edge_3_9.38_15 for training data
processing edge_3_9.38_16 for training data
processing edge_3_9.38_17 for training data
processing edge_3_9.26_0 for training data
processing edge_3_9.26_1 for training data
processing edge_3_9.26_2 for training data
processing edge_3_9.26_3 for training data
processing edge_3_9.26_4 for training data
processing edge_3_9.26_5 for training data
processing edge_3_9.26_6 for training data
processing edge_3_9.26_7 for training data
processing edge_3_9.26_8 for training data
processing edge_3_9.26_9 for training data
processing edge_3_9.26_10 for training data
processing edge_3_9.26_11 for training data
processing edge_3_9.26_12 for training data
processing edge_3_9.26_13 for training data
processing edge_3_9.26_14 for training data
processing edge_3_9.26_15 for training data
processing edge_3_9.26_16 for training dat

processing edge_3_8.22_3 for training data
processing edge_3_8.22_4 for training data
processing edge_3_8.22_5 for training data
processing edge_3_8.22_6 for training data
processing edge_3_8.22_7 for training data
processing edge_3_8.22_8 for training data
processing edge_3_8.22_9 for training data
processing edge_3_8.22_10 for training data
processing edge_3_8.22_11 for training data
processing edge_3_8.22_12 for training data
processing edge_3_8.22_13 for training data
processing edge_3_8.22_14 for training data
processing edge_3_8.22_15 for training data
processing edge_3_8.22_16 for training data
processing edge_3_8.22_17 for training data
processing edge_3_8.14_0 for training data
processing edge_3_8.14_1 for training data
processing edge_3_8.14_2 for training data
processing edge_3_8.14_3 for training data
processing edge_3_8.14_4 for training data
processing edge_3_8.14_5 for training data
processing edge_3_8.14_6 for training data
processing edge_3_8.14_7 for training data
pro

processing edge_3_7.47_12 for training data
processing edge_3_7.47_13 for training data
processing edge_3_7.47_14 for training data
processing edge_3_7.47_15 for training data
processing edge_3_7.47_16 for training data
processing edge_3_7.47_17 for training data
processing edge_3_7.41_0 for training data
processing edge_3_7.41_1 for training data
processing edge_3_7.41_2 for training data
processing edge_3_7.41_3 for training data
processing edge_3_7.41_4 for training data
processing edge_3_7.41_5 for training data
processing edge_3_7.41_6 for training data
processing edge_3_7.41_7 for training data
processing edge_3_7.41_8 for training data
processing edge_3_7.41_9 for training data
processing edge_3_7.41_10 for training data
processing edge_3_7.41_11 for training data
processing edge_3_7.41_12 for training data
processing edge_3_7.41_13 for training data
processing edge_3_7.41_14 for training data
processing edge_3_7.41_15 for training data
processing edge_3_7.41_16 for training dat

In [51]:
@time_usage
def processUnlbl(noise=0.):
    nlimPerFile = 270+nblSkip
    for run in unlblnames:
        fnames = [run+"_"+str(i) for i in uidx]
        fout = open(unlabeled_dir+run+noiseappend,'w') #erases file
        fout.close()
        for f in fnames:
            fin = open(in_dir+f,'r')
            print "processing " + f + noiseappend + " for training data"
            fout = open(unlabeled_dir+run+noiseappend,'a')

            # find width from file header
            width, height = 0., 0.
            l = fin.readline().split("|")
            for ll in l:
                if "boxEdge" in ll:
                    width = float(ll.split()[1])
            height = width
            fin.seek(0)

            if width == 0.:
                # calculate edge length based on vertices of first block
                block = []
                for line in fin.readlines():
                    if line == "\n": break
                    if line[0].isalpha(): continue
                    block.append(line)
                fin.seek(0)
                width, height = edgeLenCalc(block)

            if not (fin.readline()[0].isalpha()): fin.seek(0)

            thNorm = oneOver2Pi
            normX, normY = 1./width, 1./height # normalize x and y

            nbl = 0
            fRot = 0. # rotation factor: 0,1,2,3. Multiplied by pi/2
            block = []
            for line in fin.readlines():
                if line == "\n":
                    if nbl < nblSkip:
                        nbl+=1
                        block = []
                        continue
                    fRot = random.randint(0,3)
                    for l in block:
                        fout.write('%f %f %f\n' % (l[0], l[1], l[2]))
                    fout.write('\n')
                    block = []
                    nbl+=1
                    if nbl == nlimPerFile:
                        break
                    else:
                        continue

                rndxy = [0.,0.]
                rndth = 0.
                if noise > 0.:
                    # Gen three random numbers
                    rndxy = np.random.normal(0,noise,2)
                    rndth = np.random.normal(0,twopi*thnoise,1)
    #                 rndxy = [0.,0.]
    #                 rndth = 0.

                spt = [float(x) for x in line.split()]
                x,y,th = spt[2],spt[3],spt[4]
                # Rotate block
                # note thetas should be [0,2pi] initially
                th_ = fRot*twopi*0.25
                th += th_ + rndth
                if th > twopi: th-=twopi
                th *= thNorm

                x = np.cos(th_)*spt[2] - np.sin(th_)*spt[3] + rndxy[0]
                y = np.sin(th_)*spt[2] + np.cos(th_)*spt[3] + rndxy[1]
                # shift and normalize
                x *= normX
                y *= normY

                block.append([x,y,th])

            fout.close()
            fin.close()
    print "Done processing unlbl files" 